# First Assignment: Course of Multiagent Systems for smart machining.

Student: Daniele Trisotto

University of Trento 

## Assignment: 
Calculate the minimum and maximum Specific Cutting Pressure for each diameter, in both cases : Constant Cutting Speed and Constant Spindle Speed. 

For that, you should identify the minimum and maximum power (Pc) used in each diameter during machining, calculate the cutting force (Fc) in each case and finally the specific pressure, also called, specific cutting force (Kc).

You can present the values in a table or in a colonne graphic, for example.

## Importing Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker

# File data CSV
filedataC150 = 'ANNACHPTVC150.csv'                        #cut
filedataNC150 = 'ANNACHPTVC150VIDE.csv'                   #nocut
filedataC1200 = 'ANNACHPT1200RPM.csv'                     #cut
filedataNC1200 = 'ANNACHPT1200RPMVIDE.csv'                #nocut

# Reading csv
dfC150 = pd.read_csv(filedataC150, skiprows=10)
dfNC150 = pd.read_csv(filedataNC150, skiprows=10)
dfC1200 = pd.read_csv(filedataC1200, skiprows=10)
dfNC1200 = pd.read_csv(filedataNC1200, skiprows=10)

# # Columns names / Same for all data
dfC150.columns = [
    'time',
    'power',
    'prog_pos_z',
    'prog_pos_x',
    'meas_pos_z',
    'prog_pos_x',
    'spindle1' # only the first data does not have the spindle 2 data
]
dfNC150.columns = [
    'time',
    'power',
    'prog_pos_z',
    'prog_pos_x',
    'meas_pos_z',
    'prog_pos_x',
    'spindle1',
    'spindle2'
]
dfC1200.columns = [
    'time',
    'power',
    'prog_pos_z',
    'prog_pos_x',
    'meas_pos_z',
    'prog_pos_x',
    'spindle1',
    'spindle2'
]
dfNC1200.columns = [
    'time',
    'power',
    'prog_pos_z',
    'prog_pos_x',
    'meas_pos_z',
    'prog_pos_x',
    'spindle1',
    'spindle2'
]

cols_to_convert = dfC150.columns.difference(['time'])  

for df in [dfC150, dfNC150, dfC1200, dfNC1200]:
    for col in cols_to_convert:
        df[col] = pd.to_numeric(df[col], errors='coerce')


dfC150 = dfC150.dropna()
dfNC150 = dfNC150.dropna()
dfC1200 = dfC1200.dropna()
dfNC1200 = dfNC1200.dropna()

## Programming Parameters:

In [7]:
ap=1  #mm
f=0.2 #mm/rev
Vc150=150 #m/min

## Specific Cutting Pressure

In [ ]:
# 1) Values higher than 47 are unuseful because are starting values
dfC150_filtered = dfC150[dfC150['prog_pos_x'] <= 47].copy()

# 2) creating a mask for having only values during actual machining time
ints_mask = np.isclose(dfC150_filtered['prog_pos_x'] % 1, 0.0)
dfC150_int_positions = dfC150_filtered[ints_mask].copy()

# 3) adding a column for Kc
dfC150_int_positions['Kc']=dfC150_int_positions['power']/(Vc150*ap*f)


# 4) calculating Kc max and min
df_Kc_summary = (
    dfC150_int_positions
    .groupby('prog_pos_x')['Kc']
    .agg(['min', 'max']) # aggregate: calculates statistical functions
    .reset_index() # comes back to normal columns after a groupbyy
    .rename(columns={'min': 'Kc_min', 'max': 'Kc_max'})
)

# 5) Calculating mean Kc 
df_Kc_mean = (
    dfC150_int_positions
    .groupby('prog_pos_x')['Kc']
    .agg(['mean']) # aggregate: calculates statistical functions
)
# Mostra il risultato
print(df_Kc_mean
    )

                 mean
prog_pos_x           
24.0        75.733214
26.0        72.360571
28.0        68.769388
30.0        66.632339
32.0        65.168621
34.0        64.135660
36.0        62.974633
38.0        61.525064
40.0        60.981031
42.0        59.727203
44.0        59.002547
46.0        57.899909
